In [1]:
import numpy as np
import pinocchio as pin

In [2]:
%%capture
!pip3 install robot_descriptions
from robot_descriptions import (
    z1_mj_description,
    z1_description,
    skydio_x2_mj_description,
    skydio_x2_description,
)

In [3]:
# ROBOT_URDF = skydio_x2_description.URDF_PATH
# ROBOT_XML = skydio_x2_mj_description.MJCF_PATH
ROBOT_URDF = z1_description.URDF_PATH
ROBOT_XML = z1_mj_description.MJCF_PATH

In [4]:
model = pin.buildModelFromUrdf(ROBOT_URDF)
data = model.createData()

In [5]:
q = np.random.randn(model.nq)
if model.nq != model.nv:
    q[3:7] /= np.linalg.norm(q[3:7])
dq = np.random.randn(model.nv)
ddq = np.random.randn(model.nv)
data.q = q.copy()
data.dq = dq.copy()
data.ddq = ddq.copy()

### Regressor in Mujoco

In [6]:
import mujoco

In [7]:
# Load model, make data
mj_model = mujoco.MjModel.from_xml_path(ROBOT_XML)
mj_data = mujoco.MjData(mj_model)
mj_model.opt.enableflags |= mujoco.mjtEnableBit.mjENBL_ENERGY

# model.energy.setflags(enable=True)

# Set the state to the "squat" keyframe, call mj_forward.
# key = model.key('hover').id
# mujoco.mj_resetDataKeyframe(model, data, key)

q_muj = q.copy()
if mj_model.nq != mj_model.nv:
    q_muj[[3, 4, 5, 6]] = q[[6, 3, 4, 5]]

mj_data.qpos = q_muj
mj_data.qvel = dq.copy()
mj_data.qacc = ddq.copy()

In [8]:
mujoco.mj_inverse(mj_model, mj_data)

In [9]:
def Y_body(v_lin, v_ang, a_lin, a_ang):
    # this is identical to the pinnochio: pin.bodyRegressor(*[v_lin, v_ang], *[a_lin, a_ang])
    v1, v2, v3 = v_lin
    v4, v5, v6 = v_ang

    a1, a2, a3 = a_lin
    a4, a5, a6 = a_ang

    # fmt: off
    Y = np.array([
        [a1 - v2*v6 + v3*v5, -v5**2 - v6**2, -a6 + v4*v5, a5 + v4*v6, 0, 0, 0, 0, 0, 0],
        [a2 + v1*v6 - v3*v4, a6 + v4*v5, -v4**2 - v6**2, -a4 + v5*v6, 0, 0, 0, 0, 0, 0],
        [a3 - v1*v5 + v2*v4, -a5 + v4*v6, a4 + v5*v6, -v4**2 - v5**2, 0, 0, 0, 0, 0, 0],
        [0, 0, a3 - v1*v5 + v2*v4, -a2 - v1*v6 + v3*v4, a4, a5 - v4*v6, -v5*v6, a6 + v4*v5, v5**2 - v6**2, v5*v6],
        [0, -a3 + v1*v5 - v2*v4, 0, a1 - v2*v6 + v3*v5, v4*v6, a4 + v5*v6, a5, -v4**2 + v6**2, a6 - v4*v5, -v4*v6],
        [0, a2 + v1*v6 - v3*v4, -a1 + v2*v6 - v3*v5, 0, -v4*v5, v4**2 - v5**2, v4*v5, a4 - v5*v6, a5 + v4*v6, a6]
    ])
    # fmt: on

    return Y


def mj_bodyRegressor(mj_model, mj_data, body):
    # TODO:
    # Clarify more about transformations:
    # https://mujoco.readthedocs.io/en/stable/programming/simulation.html#coordinate-frames-and-transformations
    # https://royfeatherstone.org/teaching/index.html
    # https://royfeatherstone.org/teaching/2008/slides.pdf

    body_id = body + 1
    velocity = np.zeros(6)
    accel = np.zeros(6)
    _cross = np.zeros(3)

    mujoco.mj_objectVelocity(mj_model, mj_data, 2, body_id, velocity, 1)
    mujoco.mj_rnePostConstraint(mj_model, mj_data)
    mujoco.mj_objectAcceleration(mj_model, mj_data, 2, body_id, accel, 1)
    # rotation = mj_data.xmat[body_id].reshape(3, 3).copy()

    v, w = velocity[3:], velocity[:3]
    dv, dw = accel[3:], accel[:3]  # dv - classical acceleration, already containt g

    mujoco.mju_cross(_cross, w, v)

    dv -= _cross

    Y = Y_body(v, w, dv, dw)
    return Y


def mj_jointRegressor(mj_model, mj_data):
    # TODO:
    # what the hell is going on with indeces?
    # increase time in computation of multiplication between rotation
    # Investigate spatial tranfromation mju_transformSpatial t

    njoints = mj_model.njnt
    body_regressors = np.zeros((6 * njoints, njoints * 10))
    col_jac = np.zeros((6 * njoints, mj_model.nv))
    jac_lin = np.zeros((3, mj_model.nv))
    jac_rot = np.zeros((3, mj_model.nv))

    for i in range(njoints):
        # calculate cody regressors
        body_regressors[6 * i : 6 * (i + 1), 10 * i : 10 * (i + 1)] = mj_bodyRegressor(
            mj_model, mj_data, i + 1
        )  # regressor_body(i+1)

        mujoco.mj_jacPointAxis(
            mj_model,
            mj_data,
            jac_lin,
            None,
            mj_data.xanchor[i],
            mj_data.xaxis[i],
            i + 1,
        )
        mujoco.mj_jacBody(mj_model, mj_data, None, jac_rot, i + 2)

        # Calculate jacobians
        rotation = mj_data.xmat[i + 2].reshape(3, 3).copy()
        col_jac[6 * i : 6 * i + 3, :] = rotation.T @ jac_lin.copy()
        col_jac[6 * i + 3 : 6 * i + 6, :] = rotation.T @ jac_rot.copy()

    joint_regressor = col_jac.T @ body_regressors
    return joint_regressor

### Compare Joint regressors

In [10]:
joint_regressor_muj = mj_jointRegressor(mj_model, mj_data)

In [11]:
pin.computeAllTerms(model, data, q, dq)
pin.forwardKinematics(model, data, q, dq, ddq)
joint_regressor_pin = pin.computeJointTorqueRegressor(model, data, q, dq, ddq)

In [12]:
np.allclose(joint_regressor_muj, joint_regressor_pin)

True

### Compare Body regressors

In [13]:
body_id = 3

In [14]:
body_regressor_muj = mj_bodyRegressor(mj_model, mj_data, body_id)

In [15]:
v_spatial = pin.getVelocity(model, data, body_id, pin.LOCAL)
a_spatial = np.array(pin.getAcceleration(model, data, body_id, pin.LOCAL))
rotation = data.oMi[body_id].rotation
a_spatial[:3] -= rotation.T @ model.gravity.linear

In [16]:
body_regressor_pin = pin.bodyRegressor(pin.Motion(v_spatial), pin.Motion(a_spatial))

In [17]:
np.allclose(body_regressor_muj, body_regressor_pin)

True